<a href="https://colab.research.google.com/github/robmaz22/objects-detection/blob/main/TF_model_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Based on:
https://github.com/TannerGilbert/Tensorflow-Object-Detection-API-Train-Model

#Import of libraries

In [1]:
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET
import io
import tensorflow as tf
from PIL import Image, ImageDraw, ImageFont
from collections import namedtuple, OrderedDict
import scipy.misc
import numpy as np
import six
import time
from IPython.display import display
from six import BytesIO
import matplotlib
import matplotlib.pyplot as plt
from random import choices

In [2]:
#@title Dataset type (and name if kaggle)
from google.colab import files

dataset_type = "kaggle" #@param ["kaggle", "own"]
dataset_name = "mbkinaci/fruit-images-for-object-detection" #@param {type:"string"}

if dataset_type == 'kaggle':
  !pip -q install kaggle

  print("Send kaggle.json")
  uploaded = files.upload()  
  !mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json
  print(f"{dataset_name} downloading ...")
  !kaggle datasets download {dataset_name} --unzip

else:
  print("Send zip archive with your dataset")
  uploaded = files.upload()  

Send kaggle.json


Saving kaggle.json to kaggle.json
mbkinaci/fruit-images-for-object-detection downloading ...
 99% 28.0M/28.4M [00:01<00:00, 15.7MB/s]
100% 28.4M/28.4M [00:01<00:00, 26.7MB/s]


In [3]:

main_dir = "/content/"
train_path = f"{main_dir}train_zip/train"
test_path = f"{main_dir}test_zip/test"

labelmap_path = f'{main_dir}labelmap.pbtxt'

train_labels = f'{main_dir}{os.path.basename(train_path)}_label.csv'
test_labels = f'{main_dir}{os.path.basename(test_path)}_label.csv'
train_record_path = f'{main_dir}train.record'
test_record_path = f'{main_dir}test.record'


In [4]:
model_name = 'ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8'
fine_tune_checkpoint = f'{model_name}/checkpoint/ckpt-0'
model_dir = f'{main_dir}training/'
pipeline_config_path = f'{main_dir}model_config.config'
output_directory = f'{main_dir}inference_graph'

batch_size = 12
num_steps =  3000
num_eval_steps =  100
test_images = 3
download_model = True

#Preparation of the environment

In [5]:
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 63314, done.
remote: Counting objects: 100% (657/657), done.
remote: Compressing objects: 100% (243/243), done.
remote: Total 63314 (delta 513), reused 554 (delta 414), pack-reused 62657
Receiving objects: 100% (63314/63314), 574.71 MiB | 23.27 MiB/s, done.
Resolving deltas: 100% (44186/44186), done.


In [6]:
%cd models/research
!protoc object_detection/protos/*.proto --python_out=.
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .
!python object_detection/builders/model_builder_tf2_test.py

/content/models/research
Processing /content/models/research
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 9.8 MB 16.3 MB/s 
     |████████████████████████████████| 352 kB 43.2 MB/s 
     |████████████████████████████████| 1.8 MB 40.0 MB/s 
     |████████████████████████████████| 211 kB 47.3 MB/s 
     |████████████████████████████████| 37.1 MB 48 kB/s 
     |████████████████████████████████| 43 kB 2.0 MB/s 
     |████████████████████████████████| 4.4 MB 40.3 MB/s 
     |████████████████████████████████| 636 kB 44.8 MB/s 
     |████████████████████████████████| 90 kB 10.3 MB/s 
     |███████████████████

In [7]:
from object_detection.utils import dataset_util
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

#Labels conversion from xml to TFRecord format.

In [8]:
def xml_to_csv(path):
    xml_values = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text)
                     )
            xml_values.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_values, columns=column_name)
    return xml_df

In [9]:
for path in [train_path, test_path]:
  image_path = path
  xml_df = xml_to_csv(image_path)
  xml_df.to_csv(f'{main_dir}{os.path.basename(path)}_label.csv', index=None)

In [10]:
df = pd.read_csv(train_labels)
class_names = list(set(df['class']))

with open(labelmap_path, 'w') as f:
  for i, name in enumerate(class_names):
    content = "item {\n"
    content += f"    id: {i+1}\n"
    content += f"    name: '{name}'\n"
    content += "}\n"
    f.write(content)

In [11]:
def class_text_to_int(row_label):
  for i, name in enumerate(class_names):
    if row_label == name:
        return i + 1
    

def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


def create_tf_example(group, path):
    with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

In [12]:
for content in [(train_record_path, train_path, train_labels), (test_record_path, test_path, test_labels)]:
  writer = tf.io.TFRecordWriter(content[0])
  path = content[1]
  examples = pd.read_csv(content[2])
  grouped = split(examples, 'filename')
  for group in grouped:
        tf_example = create_tf_example(group, path)
        writer.write(tf_example.SerializeToString())
  writer.close()
  output_path = os.path.join(content[0])

#Model download and configuration.

In [13]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/{model_name}.tar.gz
!tar -xf {model_name}.tar.gz

--2021-09-25 11:14:27--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 108.177.15.128, 2a00:1450:400c:c0c::80
Connecting to download.tensorflow.org (download.tensorflow.org)|108.177.15.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 90453990 (86M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8.tar.gz’

ssd_mobilenet_v1_fp 100%[===================>]  86.26M  68.0MB/s    in 1.3s    

2021-09-25 11:14:28 (68.0 MB/s) - ‘ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8.tar.gz’ saved [90453990/90453990]



In [14]:
!wget https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/{model_name}.config

base_config_path = f'{model_name}.config'

--2021-09-25 11:14:30--  https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8.config
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4528 (4.4K) [text/plain]
Saving to: ‘ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8.config’

ssd_mobilenet_v1_fp 100%[===================>]   4.42K  --.-KB/s    in 0s      

2021-09-25 11:14:30 (39.2 MB/s) - ‘ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8.config’ saved [4528/4528]



In [15]:
import re

with open(base_config_path) as f:
    config = f.read()

with open(pipeline_config_path, 'w') as f:
  
  config = re.sub('label_map_path: ".*?"', 
             'label_map_path: "{}"'.format(labelmap_path), config)
  
  config = re.sub('fine_tune_checkpoint: ".*?"',
                  'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), config)
  
  config = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 
                  'input_path: "{}"'.format(train_record_path), config)
  
  config = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 
                  'input_path: "{}"'.format(test_record_path), config)
  
  config = re.sub('num_classes: [0-9]+',
                  'num_classes: {}'.format(3), config)
  
  config = re.sub('batch_size: [0-9]+',
                  'batch_size: {}'.format(batch_size), config)

  config = re.sub('num_steps: [0-9]+',
                  'num_steps: {}'.format(num_steps), config)

  config = re.sub('fine_tune_checkpoint_type: "classification"', 
             'fine_tune_checkpoint_type: "{}"'.format('detection'), config)
  
  f.write(config)

#Training

In [ ]:
!python /content/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_config_path} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps}\
    --sample_1_of_n_eval_examples=1 \
    --num_eval_steps={num_eval_steps}

2021-09-25 11:14:33.842111: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-25 11:14:33.852149: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-25 11:14:33.852963: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-25 11:14:33.854532: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-25 11:14:33.855379: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

#Training history

In [ ]:
%load_ext tensorboard
%tensorboard --logdir {model_dir}/train

#Model eksport

In [ ]:
!python /content/models/research/object_detection/exporter_main_v2.py \
    --trained_checkpoint_dir {model_dir} \
    --output_directory {output_directory} \
    --pipeline_config_path {pipeline_config_path}

#Model accuracy

In [ ]:
def load_image_into_numpy_array(path):
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(labelmap_path, use_display_name=True)

In [ ]:
tf.keras.backend.clear_session()
model = tf.saved_model.load(f'{output_directory}/saved_model')

In [ ]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  input_tensor = tf.convert_to_tensor(image)
  input_tensor = input_tensor[tf.newaxis,...]

  model_fn = model.signatures['serving_default']
  output_dict = model_fn(input_tensor)

  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)

  if 'detection_masks' in output_dict:
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

In [ ]:
train_set = glob.glob(f'{test_path}/*.jpg')

for image_path in choices(train_set, k=test_images):
  print(image_path)
  image_np = load_image_into_numpy_array(image_path)
  output_dict = run_inference_for_single_image(model, image_np)
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=8)
  display(Image.fromarray(image_np))

#Model and labels download.

In [ ]:
from google.colab import files

if download_model:
  print("Download files")
  !zip -r saved_model.zip {output_directory}/saved_model
  files.download(f'saved_model.zip')
  files.download(f'{labelmap_path}')